## USE GPU

In [ ]:
!pip install transformers
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Loading Mutilingual BERT





In [ ]:
# Install SentencePiece
!pip install sentencepiece

# Now you can import the tokenizer and model
from transformers import XLMRobertaTokenizer, XLMRobertaModel

# Load tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-large")
model = XLMRobertaModel.from_pretrained("xlm-roberta-large")


In [ ]:
def get_bert_embeddings(text):
    # Ensure text is a string
    text = str(text)
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1)

# Assume text1 and text2 are your text data in Spanish or English
text1 = "This is a sample text in English."
text2 = "Este es un texto de ejemplo en español."

# Get BERT embeddings
embedding1 = get_bert_embeddings(text1)
embedding2 = get_bert_embeddings(text2)


In [ ]:
# Convert torch tensors to numpy arrays for use with sklearn
embedding1_np = embedding1.numpy()
embedding2_np = embedding2.numpy()

# Compute cosine similarity
similarity = cosine_similarity(embedding1_np, embedding2_np)
print(f'Cosine Similarity: {similarity[0][0]}')
print(similarity)

Cosine Similarity: 0.99785315990448
[[0.99785316]]


Let's Start Building

##Getting the Data

In [ ]:
import zipfile
!wget https://www.dropbox.com/s/yle22nlwxhgf7pb/color.zip?dl=0&file_subpath=%2Fcolor%2Fexperiment1-dataset-color-of-concrete-objects.txt
# Specify the name of the ZIP file
zip_file = './color.zip?dl=0'

# Specify the directory where you want to extract the contents
extract_path = './'

# Create the directory if it doesn't exist
import os
os.makedirs(extract_path, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f'Contents of {zip_file} have been extracted to {extract_path}')
%cd color



--2023-11-05 17:13:58--  https://www.dropbox.com/s/yle22nlwxhgf7pb/color.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/yle22nlwxhgf7pb/color.zip [following]
--2023-11-05 17:13:58--  https://www.dropbox.com/s/raw/yle22nlwxhgf7pb/color.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc638d076019aab837ca12455e15.dl.dropboxusercontent.com/cd/0/inline/CG8x7S3aznpRe2T4S1Qj-qwgIp9voGQDdXjzur9qsDx9lMlBphpAYpnGeJsGG7uVQ_RC1PhjPOb7uUQ6rxxp-5XFHqxFKdCmNem1irViQxSavc66oftKTh2Lt7vB92tVifs/file# [following]
--2023-11-05 17:13:58--  https://uc638d076019aab837ca12455e15.dl.dropboxusercontent.com/cd/0/inline/CG8x7S3aznpRe2T4S1Qj-qwgIp9voGQDdXjzur9qsDx9lMlBphpAYpnGeJsGG7uVQ_RC1PhjPOb7uUQ6rxxp-5XFHqxFKdCmNem1irViQxSavc66oftKTh2Lt7vB92t

In [ ]:
import pandas as pd
df = pd.read_csv('experiment1-dataset-color-of-concrete-objects.txt',sep='\t', names =['noun','color'])
bless = df

In [ ]:
# Add true_similarity column
def compute_cosine_similarity(noun, color):
    noun_vector = get_bert_embeddings(noun)
    color_vector = get_bert_embeddings(color)
    noun_vector_np = noun_vector.numpy()
    color_vector_np = color_vector.numpy()
    similarity = cosine_similarity(noun_vector_np, color_vector_np)
    return similarity[0][0]

bless['true_similarity'] = bless.apply(lambda row: compute_cosine_similarity(row['noun'], row['color']), axis=1)

color_list = ['black','blue','brown','green','grey','orange','pink','purple','red','white','yellow']

# For each color, compute the cosine similarity and store in a new column
for color in color_list:
    bless[color] = bless['noun'].apply(lambda noun: compute_cosine_similarity(noun, color))

bless['Max_similarity'] = bless[color_list].max(axis=1)
bless['Max_color'] = bless[color_list].idxmax(axis=1)
bless['accuracy'] = (bless['color'] == bless['Max_color']).astype(int)
bless

,noun,color,true_similarity,black,blue,brown,green,grey,orange,pink,purple,red,white,yellow,Max_similarity,Max_color,accuracy
0,ash,grey,0.994844,0.991646,0.992233,0.992048,0.992458,0.994844,0.994121,0.992367,0.994303,0.991231,0.992238,0.990490,0.994844,grey,1
1,banana,yellow,0.998260,0.998353,0.998474,0.998661,0.998535,0.994111,0.996430,0.998038,0.995508,0.998487,0.998383,0.998260,0.998661,brown,0
2,beaver,brown,0.995090,0.994863,0.994658,0.995090,0.995117,0.995971,0.996697,0.994966,0.995560,0.994252,0.995106,0.993983,0.996697,orange,0
3,blood,red,0.998947,0.998705,0.998919,0.998721,0.998389,0.994327,0.996570,0.998353,0.995883,0.998947,0.998751,0.998387,0.998947,red,1
4,broccoli,green,0.991426,0.991311,0.990792,0.991210,0.991426,0.992566,0.992582,0.991109,0.992081,0.990284,0.991534,0.989350,0.992582,orange,0
5,cardboard,brown,0.996945,0.997058,0.996840,0.996945,0.997336,0.995138,0.996448,0.997129,0.996031,0.996509,0.997377,0.996077,0.997377,white,0
6,carrot,orange,0.996630,0.997880,0.997645,0.997948,0.997585,0.994471,0.996630,0.997585,0.995731,0.997655,0.997740,0.997090,0.997948,brown,0
7,cauliflower,white,0.995338,0.995301,0.995052,0.995113,0.995765,0.993823,0.995320,0.995265,0.995267,0.994352,0.995338,0.993831,0.995765,green,0
8,cello,brown,0.998060,0.997975,0.998030,0.998060,0.998084,0.994772,0.996786,0.998043,0.996552,0.997661,0.998114,0.996659,0.998114,white,0
9,chalk,white,0.994879,0.994635,0.994990,0.994752,0.995101,0.996024,0.996503,0.995182,0.995972,0.993974,0.994879,0.993731,0.996503,orange,0


In [ ]:
overall_accuracy = bless['accuracy'].sum() / len(bless)

print(f"Overall Accuracy: {overall_accuracy:.2%}")

Overall Accuracy: 13.46%


In [ ]:
# For each row, rank the cosine similarities and determine the rank of the true color
ranks = []
for index, row in bless.iterrows():
    ranked_colors = row[color_list].sort_values(ascending=False).index.tolist()
    rank_of_true_color = ranked_colors.index(row['color']) + 1
    ranks.append(rank_of_true_color)

# Add ranks to the dataframe
bless['rank'] = ranks

# Calculate median rank
median_rank = pd.Series(ranks).median()

print("Median Rank:", median_rank)

Median Rank: 4.0


In [ ]:
!pip install deep_translator
from deep_translator import GoogleTranslator
to_translate = 'I want to translate this text'
def translate_text(text):
  return GoogleTranslator(source='auto', target='es').translate(text)
# outpout -> Ich möchte diesen Text übersetzen
#from googletrans import Translator

def translate_column_to_spanish(df, column_name1, column_name2):
    # Initialize the translator


    # Translate the words in the specified column to Spanish
    df['nombre_esp'] = df[column_name1].apply(lambda text: translate_text(text,))
    df['color_esp'] = df[column_name2].apply(lambda text: translate_text(text,))

    return df


# Call the translate_column_to_spanish function
translated_df = translate_column_to_spanish(df, 'noun', 'color')


# Display the translated DataFrame
print(translated_df)
bless = translated_df




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00
           noun   color  true_similarity     black      blue     brown  \
0           ash    grey         0.994844  0.991646  0.992233  0.992048   
1        banana  yellow         0.998260  0.998353  0.998474  0.998661   
2        beaver   brown         0.995090  0.994863  0.994658  0.995090   
3         blood     red         0.998947  0.998705  0.998919  0.998721   
4      broccoli   green         0.991426  0.991311  0.990792  0.991210   
5     cardboard   brown         0.996945  0.997058  0.996840  0.996945   
6        carrot  orange         0.996630  0.997880  0.997645  0.997948   
7   cauliflower   white         0.995338  0.995301  0.995052  0.995113   
8         cello   brown         0.998060  0.997975  0.998030  0.998060   
9         chalk   white         0.994879  0.994635  0.994990  0.994752   
10       cherry     red         0.996894  0.996772  0.996688  0.996861   
11        cloud   white         

In [ ]:
print(bless['nombre_esp'].isnull().sum())
print(bless['color_esp'].isnull().sum())

0
0


In [ ]:
embedding1 = get_bert_embeddings(text1)
embedding2 = get_bert_embeddings(text2)

def compute_cosine_similarity(noun, color):
    noun_vector = get_bert_embeddings(noun)
    color_vector = get_bert_embeddings(color)
    noun_vector_np = noun_vector.numpy()
    color_vector_np = color_vector.numpy()
    similarity = cosine_similarity(noun_vector_np, color_vector_np)
    return similarity[0][0]

# Add true_similarity column
bless['true_similarity'] = bless.apply(lambda row: compute_cosine_similarity(row['nombre_esp'], row['color_esp']), axis=1)

color_list = ['gris', 'amarillo', 'marrón', 'rojo', 'verde', 'naranja', 'blanco', 'negro', 'rosa', 'azul']

# For each color, compute the cosine similarity and store in a new column
for color in color_list:
    bless[color] = bless['nombre_esp'].apply(lambda noun: compute_cosine_similarity(noun, color))

bless['Max_similarity'] = bless[color_list].max(axis=1)
bless['Max_color'] = bless[color_list].idxmax(axis=1)
bless['accuracy'] = (bless['color_esp'] == bless['Max_color']).astype(int)
bless

,noun,color,true_similarity,black,blue,brown,green,grey,orange,pink,...,gris,amarillo,marrón,rojo,verde,naranja,blanco,negro,rosa,azul
0,ash,grey,0.994926,0.991646,0.992233,0.992048,0.992458,0.994844,0.994121,0.992367,...,0.994926,0.995047,0.995270,0.995090,0.995031,0.993930,0.992857,0.994939,0.994771,0.992876
1,banana,yellow,0.995540,0.998353,0.998474,0.998661,0.998535,0.994111,0.996430,0.998038,...,0.997921,0.995540,0.995958,0.997258,0.998304,0.993916,0.997411,0.997494,0.998298,0.997449
2,beaver,brown,0.995494,0.994863,0.994658,0.995090,0.995117,0.995971,0.996697,0.994966,...,0.994946,0.995239,0.995494,0.996135,0.995695,0.995016,0.993814,0.995420,0.995435,0.993785
3,blood,red,0.997521,0.998705,0.998919,0.998721,0.998389,0.994327,0.996570,0.998353,...,0.997675,0.995723,0.996450,0.997521,0.997965,0.994126,0.996516,0.997847,0.997992,0.996593
4,broccoli,green,0.996997,0.991311,0.990792,0.991210,0.991426,0.992566,0.992582,0.991109,...,0.996718,0.996596,0.996772,0.997411,0.996997,0.995051,0.995085,0.996943,0.996789,0.995051
5,cardboard,brown,0.994719,0.997058,0.996840,0.996945,0.997336,0.995138,0.996448,0.997129,...,0.994333,0.994821,0.994719,0.995573,0.994585,0.993974,0.993029,0.994705,0.994494,0.993117
6,carrot,orange,0.995123,0.997880,0.997645,0.997948,0.997585,0.994471,0.996630,0.997585,...,0.993551,0.994723,0.994309,0.994493,0.994043,0.995123,0.992482,0.994204,0.994157,0.992462
7,cauliflower,white,0.994804,0.995301,0.995052,0.995113,0.995765,0.993823,0.995320,0.995265,...,0.996442,0.996709,0.996347,0.997124,0.996903,0.994766,0.994804,0.996870,0.996516,0.994847
8,cello,brown,0.986183,0.997975,0.998030,0.998060,0.998084,0.994772,0.996786,0.998043,...,0.985150,0.986337,0.986183,0.987152,0.986643,0.989803,0.984274,0.986100,0.985680,0.984336
9,chalk,white,0.993304,0.994635,0.994990,0.994752,0.995101,0.996024,0.996503,0.995182,...,0.994802,0.994047,0.994904,0.995548,0.995172,0.994993,0.993304,0.994890,0.995052,0.993400


In [ ]:
overall_accuracy = bless['accuracy'].sum() / len(bless)

print(f"Overall Accuracy: {overall_accuracy:.2%}")

Overall Accuracy: 7.69%


In [ ]:
# For each row, rank the cosine similarities and determine the rank of the true color
ranks = []
for index, row in bless.iterrows():
    ranked_colors = row[color_list].sort_values(ascending=False).index.tolist()
    rank_of_true_color = ranked_colors.index(row['color_esp']) + 1
    ranks.append(rank_of_true_color)

# Add ranks to the dataframe
bless['rank'] = ranks

# Calculate median rank
median_rank = pd.Series(ranks).median()

print("Median Rank:", median_rank)

Median Rank: 5.0
